In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

zsh:1: command not found: apt-get


In [2]:
!wget https://archive.apache.org/dist/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
!tar xf spark-3.5.3-bin-hadoop3.tgz
!rm spark-3.5.3-bin-hadoop3.tgz   # Tidying up

--2025-03-04 10:24:54--  https://archive.apache.org/dist/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189
connected. to archive.apache.org (archive.apache.org)|65.108.204.189|:443... 
HTTP request sent, awaiting response... 200 OK
Length: 400864419 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.3-bin-hadoop3.tgz’

spark-3.5.3-bin-had 100%[===================>] 382.29M   255KB/s    in 33m 32s 

2025-03-04 10:58:26 (195 KB/s) - ‘spark-3.5.3-bin-hadoop3.tgz’ saved [400864419/400864419]



In [1]:
import os
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk-21.jdk/Contents/Home"
os.environ["SPARK_HOME"] = "spark-3.5.3-bin-hadoop3"

In [2]:
!cd ~/CS/CovidComparison
!pwd

/Users/duncanbeauchamp/CS/CovidComparison


In [3]:
# !pip install -q findspark
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) #  This will format our output tables a bit nicer when not using the show() method
spark

25/03/20 10:15:05 WARN Utils: Your hostname, Duncans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.18.95.63 instead (on interface en0)
25/03/20 10:15:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/20 10:15:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
import multiprocessing
print(multiprocessing.cpu_count())

8


25/03/20 10:15:18 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [6]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setAppName("CSV Loader").setMaster("local[*]")
sc = SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by getOrCreate at /var/folders/91/t1fb1wxx11x5nxs1h30387tw0000gn/T/ipykernel_68216/3781185930.py:2 

In [10]:
df = spark.read.csv("reddit_comments.csv", header=True, inferSchema=True, multiLine=True, escape='"')


def parse_csv_line(line):
    import csv
    from io import StringIO

    try:
        reader = csv.reader(StringIO(line))
        parsed = next(reader, None)  # Use default `None` to avoid StopIteration
        return parsed if parsed else []  # Return empty list instead of None
    except Exception as e:
        print(f"Error parsing line: {line}, Error: {e}")
        return []  # Return empty list to filter out later

rdd_parsed = rdd.map(parse_csv_line).filter(lambda x: x)  # Filter out empty lists

print(rdd_parsed.take(5))  # Check the first 5 rows

Py4JJavaError: An error occurred while calling o67.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/Users/duncanbeauchamp/CS/CovidComparison/path/to/reddit_comments.csv
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.api.java.JavaRDDLike

In [6]:
small_reddit.take(5)

['id,subreddit.name,created_utc,body,score',
 'hi1vsag,nova,1635206399,When you scheduled your booster with CVS does it just give you the option of Vaccines: COVID-19 (Vaccine brand) or does it specifically say booster,2',
 'hi1vs7i,vancouver,1635206397,"Didn\'t stop prices there though. New Zealand and Canada grew at about the same rate through COVID. ',
 '',
 'I agree that non-resident ownership should be stopped, but it\'s also probably not going to change anything.",32']

In [7]:
reddit_df = spark.read.csv("small_reddit.csv", header=True, inferSchema=True, multiLine=True, escape='"')

reddit_df.show()


+-------+--------------------+-----------+--------------------+-----+
|     id|      subreddit.name|created_utc|                body|score|
+-------+--------------------+-----------+--------------------+-----+
|hi1vsag|                nova| 1635206399|When you schedule...|    2|
|hi1vs7i|           vancouver| 1635206397|Didn't stop price...|   32|
|hi1vs5n|            pregnant| 1635206397|I’m just waiting ...|    1|
|hi1vs5v|            startrek| 1635206397|*The first duty o...|    1|
|hi1vs0l|             ontario| 1635206395|Compare BC to Ont...|   -2|
|hi1vrtk|       entertainment| 1635206393|Sheeran didn't me...|    2|
|hi1vrsk|          conspiracy| 1635206392|I see a lot of po...|    1|
|hi1vrnr|  lockdownskepticism| 1635206391|And think about h...|   29|
|hi1vrpy|                nova| 1635206391|Unfortunately the...|    2|
|hi1vrou|         antivaxxers| 1635206391|I don’t know anyo...|    1|
|hi1vrg2|          conspiracy| 1635206388|BS. Not a fear ca...|    1|
|hi1vre5|      toron

In [13]:
from pyspark.sql.functions import regexp_replace

# Assume df is your DataFrame with a "comment" column containing newlines
reddit_df_cleaned = reddit_df.withColumn("body", regexp_replace("body", "[\r\n]", " "))

reddit_df_cleaned.take(2)


[Row(id='hi1vsag', subreddit.name='nova', created_utc=1635206399, body='When you scheduled your booster with CVS does it just give you the option of Vaccines: COVID-19 (Vaccine brand) or does it specifically say booster', score=2),
 Row(id='hi1vs7i', subreddit.name='vancouver', created_utc=1635206397, body="Didn't stop prices there though. New Zealand and Canada grew at about the same rate through COVID.   I agree that non-resident ownership should be stopped, but it's also probably not going to change anything.", score=32)]

In [16]:
df_cleaned.coalesce(1).write.csv("./test", header=True)

In [17]:
twitter_df = spark.read.csv("small_twitter.csv", header=True, inferSchema=True, multiLine=True, escape='"')

twitter_df.show()

+-------------------+-------------------+---------------+--------------------+----+--------------------+-------------+--------------------+------------+---------------+--------+
|          status_id|         created_at|    screen_name|                text|lang|              source|retweet_count|reply_to_screen_name|country_code|followers_count|verified|
+-------------------+-------------------+---------------+--------------------+----+--------------------+-------------+--------------------+------------+---------------+--------+
|1244051646071611394|2020-03-29 00:00:00|   IMSS_SanLuis|Ante cualquier en...|  es|           TweetDeck|            0|                NULL|        NULL|           1008|   false|
|1244051645039706112|2020-03-29 00:00:00|     intrac_ccs|#ATENCIÓN En el T...|  es|           TweetDeck|            1|                NULL|        NULL|             90|   false|
|1244051645975191557|2020-03-29 00:00:00|       rlieving|“People are just ...|  en|           TweetDeck|      

In [20]:
twitter_df_cleaned = twitter_df.withColumn("text", regexp_replace("text", "[\r\n]", " "))

twitter_df_cleaned.take(2)

[Row(status_id=1244051646071611394, created_at=datetime.datetime(2020, 3, 29, 0, 0), screen_name='IMSS_SanLuis', text='Ante cualquier enfermedad respiratoria, no te automediques. #PrevenciónCoronavirus #Coronavirus #COVID19 https://t.co/IgB2izYi1p', lang='es', source='TweetDeck', retweet_count=0, reply_to_screen_name=None, country_code=None, followers_count=1008, verified=False),
 Row(status_id=1244051645039706112, created_at=datetime.datetime(2020, 3, 29, 0, 0), screen_name='intrac_ccs', text='#ATENCIÓN En el Terminal Nuevo Circo se implementan medidas como el uso de la mascarilla por parte de usuari@s y conductores que hacen vida en dichas instalaciones, parte de los esfuerzos unificados para impedir la propagación del #Coronavirus #28Marzo @NicolasMaduro  @ErikaPSUV https://t.co/yD8g0EAcQz', lang='es', source='TweetDeck', retweet_count=1, reply_to_screen_name=None, country_code=None, followers_count=90, verified=False)]

In [23]:
twitter_df_cleaned.coalesce(1).write.csv("./twitter_test", header=True)